<a href="https://colab.research.google.com/github/geordee/colab/blob/main/HRMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HRMS Data Analytics
Analyze HRMS data using Python and Pandas
Pandas user guide is available at<br />
https://pandas.pydata.org/docs/user_guide/index.html#user-guide

Import required data analysis libraries

In [ ]:
!pip install -q xlrd
import numpy as np
import pandas as pd


Read Excel file data as a Dataframe

In [ ]:
employee_df = pd.read_excel("HRMS.xlsx", sheet_name="Employees")
employee_df.head()

Add JoinYear as a new column using a transformation on JoinDate

In [ ]:
employee_df['JoinDate'] = pd.to_datetime(employee_df['JoinDate'])
employee_df['JoinYear'] = employee_df['JoinDate'].dt.year
employee_df.head()

Pivot table on Department and Band, and aggregate (count)

In [ ]:
employee_df.pivot_table(index=['Department'], columns='Band', values='EmployeeID', aggfunc='count').fillna('')

Filter the table and pivot

In [ ]:
employee_df[employee_df.Department == "Research and Development"].pivot_table(index=['Department'], columns='Band', values='EmployeeID', aggfunc='count').fillna(0)

## Compute Bonus

In [ ]:
bonus_df = employee_df.loc[:, ["EmployeeID"]]
bonus_df.head()

In [ ]:
salary_df = pd.read_excel("HRMS.xlsx", sheet_name="Salaries")
salary_df.head()

In [ ]:
bonus_df = pd.merge(bonus_df, 
                    salary_df[['EmployeeID','Salary']], 
                    on = 'EmployeeID', 
                    how = 'inner')
bonus_df.head()

In [ ]:
bonus_df = pd.merge(bonus_df, 
                    employee_df[['EmployeeID','Band']], 
                    on = 'EmployeeID', 
                    how = 'inner')
bonus_df.head()

In [ ]:
ratings_df = pd.read_excel("HRMS.xlsx", sheet_name="Ratings")
ratings_df.head()

In [ ]:
bonus_df = pd.merge(bonus_df, 
                    ratings_df[['EmployeeID','Rating']], 
                    on = 'EmployeeID', 
                    how = 'inner')
bonus_df.head()

In [ ]:
settings_df = pd.read_excel("HRMS.xlsx", sheet_name="Settings")
settings_df.head()

In [ ]:
bonus_df = pd.merge(bonus_df, 
                    settings_df[['Band','Rating','Bonus %']], 
                    on = ['Band','Rating'] ,
                    how = 'inner')
bonus_df.head()

In [ ]:
bonus_df['Bonus']=bonus_df['Salary']*2*bonus_df['Bonus %']
bonus_df.head()


In [ ]:
bonus_df['Bonus']=np.maximum(bonus_df['Bonus'],5000)
bonus_df.head()